In [1]:
import os
import mysql.connector
import pandas as pd

In [2]:
host = os.environ['MYSQL_HOST']
user = os.environ['MYSQL_USER']
password = os.environ['MYSQL_PASSWORD']
database = os.environ['MYSQL_DB']

In [3]:
# database connector
cnx = mysql.connector.connect(user=user, password=password,
                              host=host,
                              database=database)



1. How many movies are in data set ?

In [4]:
sql_query = 'select count(distinct movieId, title) from movies_dataset.movies'
print(f'There are {pd.read_sql(sql_query,cnx).values[0]} movies in dataset.')

There are [9742] movies in dataset.


2. What is the most common genre of movie?

In [5]:
sql_query = 'select genres, count(*) from movies_dataset.movies group by genres'
df = pd.read_sql(sql_query,cnx)
display(df.head())
#clean data
df.genres = df.genres.str.replace('\r', '')
display(df.head())
#split genres column
genres_splitted = df.genres.str.split(pat = '|', expand = True)
display('column genres splitted: ', genres_splitted.head())

,genres,count(*)
0,(no genres listed)\r,34
1,Action\r,60
2,Action|Adventure\r,32
3,Action|Adventure|Animation\r,18
4,Action|Adventure|Animation|Children\r,6


,genres,count(*)
0,(no genres listed),34
1,Action,60
2,Action|Adventure,32
3,Action|Adventure|Animation,18
4,Action|Adventure|Animation|Children,6


'column genres splitted: '

,0,1,2,3,4,5,6,7,8,9
0,(no genres listed),None,None,None,None,None,None,None,None,None
1,Action,None,None,None,None,None,None,None,None,None
2,Action,Adventure,None,None,None,None,None,None,None,None
3,Action,Adventure,Animation,None,None,None,None,None,None,None
4,Action,Adventure,Animation,Children,None,None,None,None,None,None


In [6]:
display("VALUE COUNTS in first column:",df.genres.str.split(pat = '|', expand = True)[0].value_counts())

'VALUE COUNTS in first column:'

Action                341
Adventure             173
Comedy                110
Animation              79
Drama                  75
Crime                  55
Children               40
Fantasy                17
Horror                 16
Documentary            13
Mystery                 7
Film-Noir               6
Romance                 6
Musical                 5
Sci-Fi                  4
War                     1
(no genres listed)      1
Thriller                1
Western                 1
Name: 0, dtype: int64

In [7]:
melted = genres_splitted.melt().value
display("VALUE COUNTS over all columns:",melted.value_counts())

'VALUE COUNTS over all columns:'

Drama                 349
Comedy                341
Action                341
Adventure             310
Thriller              274
Fantasy               264
Sci-Fi                236
Animation             200
Romance               200
Crime                 181
Children              174
Mystery               166
Horror                153
Musical                96
IMAX                   91
War                    75
Western                64
Film-Noir              30
Documentary            30
(no genres listed)      1
Name: value, dtype: int64

There is no straight answer to this question. If we assume that only first category assigned to the movie is main category then the action movies is the largest group in dataset. But if we assume that all categories assigned to the movie describe them equally then the drama movies is the most popular category.

3. What are top 10 movies with highest rate ?

In [8]:
sql_query = '''
select m.title, sum(r.rating) as rating
from movies_dataset.ratings r
left join  movies_dataset.movies m on r.movieId = m.movieId 
group by m.title
order by rating DESC
limit 10
'''
df = pd.read_sql(sql_query,cnx)
display('TOP 10 MOVIES:', df)

'TOP 10 MOVIES:'

,title,rating
0,Forrest Gump (1994),78.0
1,Raiders of the Lost Ark (Indiana Jones and the...,62.0
2,Gladiator (2000),58.0
3,"Silence of the Lambs, The (1991)",56.0
4,Pulp Fiction (1994),56.0
5,"Shawshank Redemption, The (1994)",56.0
6,Star Wars: Episode IV - A New Hope (1977),54.0
7,Seven (a.k.a. Se7en) (1995),52.0
8,"Matrix, The (1999)",51.0
9,Schindler's List (1993),49.0


4. What are 5 most often rating users ?

*Comment: I understand this as users with the highest number of given rates.

In [9]:
sql_query = '''
select r.userId, count(r.rating) as number_of_ratings
from movies_dataset.ratings r
group by r.userId
order by number_of_ratings desc
limit 5
'''
df = pd.read_sql(sql_query,cnx)
display('TOP 5 ACTIVE USERS:', df)

'TOP 5 ACTIVE USERS:'

,userId,number_of_ratings
0,19,703
1,18,502
2,21,443
3,6,314
4,28,299


When was done first and last rate included in data set and what was the rated movie tittle?

In [10]:
sql_query = '''
select from_unixtime(min(timestamps)) as first_date, from_unixtime(max(timestamps)) as last_date from ratings;
'''
df = pd.read_sql(sql_query,cnx)
df.head()

,first_date,last_date
0,1996-06-22 10:56:16,2018-08-28 15:35:34


In [11]:
sql_query = '''
select m.title, from_unixtime(r.timestamps) as timestamps
from movies_dataset.ratings r
left join  movies_dataset.movies m on r.movieId = m.movieId 
where r.timestamps = (select min(timestamps) from ratings);
'''
df = pd.read_sql(sql_query,cnx)
print('First rate')
df.head()

First rate


,title,timestamps
0,Batman (1989),1996-06-22 10:56:16


In [12]:
sql_query = '''
select m.title, from_unixtime(r.timestamps)
from movies_dataset.ratings r
left join  movies_dataset.movies m on r.movieId = m.movieId 
where r.timestamps = (select max(timestamps) from ratings);
'''
df = pd.read_sql(sql_query,cnx)
print('Last rate')
df.head()

Last rate


,title,from_unixtime(r.timestamps)
0,Deadpool 2 (2018),2018-08-28 15:35:34
